Version 1 d'un k-means aléatoire basé sur la distance à la centroïde, calcul des distances pas opti donc à refaire, + possibilité de paralléliser facilement (calcul des différentes distances en parallèle)

In [51]:
import numpy as np
S = np.zeros((4,4))
S[0, :]


array([0., 0., 0., 0.])

In [52]:
import numpy as np

def kmeans(nodes, k):
    Keys = list(nodes.keys())
    Keys = list(map(lambda t : int(t), Keys))
    
    #------ kmeans++ -------
    # choose the centroids according to the distance between the already chosen centroids
    def selection(Keys, k):
        centroids = np.zeros(k)
        centroids[0] = np.random.choice(Keys)
        S = np.zeros((k, len(Keys)))

        for j in range(len(Keys)):
            S[0,j] = dist([centroids[0]], [Keys[j]], nodes, [], [], 0)

        dens_vect = S[0, :]
        dens_vect /= np.sum(dens_vect)
        for i in range(1, k):
            centroids[i] = np.random.choice(Keys, p = dens_vect)
            S[i,j] = dist([centroids[i]], [Keys[j]], nodes, [], [], 0)
            
            for j in range(len(dens_vect)):
                dens_vect[j] += S[i,j]
            
            dens_vect /= np.sum(dens_vect)
        return S, centroids
        

    #-----------------------

    
    # fonction pour clusteriser

    def dist(nodes1, nodes2, nodes, list1, list2, iter):
        #remonter l'arbre jusqu'à que l'intersection ne soit plus vide
        inter = list(set([i[0] for i in list1]) & set([i[0] for i in list2]))
        if len(inter) != 0:
            m1 = np.inf
            for i in list1:
                if i[0] == inter[0] and i[1] < m1:
                    m1 = i[1]
            m2 = np.inf
            for j in list2:
                if j[0] == inter[0] and j[1] < m2:
                    m2 = j[1]
            return m1 + m2
        
        else:
            list1 += [(i, iter) for i in nodes1]
            list2 += [(i, iter) for i in nodes2]
            n1, n2 = [], []
            for node in nodes1:
                n1 += nodes[str(int(node))]['Dependencies']
            for node in nodes2:
                n2 += nodes[str(int(node))]['Dependencies']
            return dist(n1, n2, nodes, list1, list2, iter + 1)

    #compute the distance between each point and centroid

    #directly computes S with only centroids,
    """
    def dist_cen(centroid, node, iter, limit): 
        to_visit = Keys
        while len(to_visit) > 0:
            """
    
    S, centroids = selection(Keys, k)
    keys_cen = [i for i in centroids]


    # clustering

    for key in Keys:
        j = Keys.index(key)
        m = np.inf
        cluster = np.inf
        for c in centroids:
            i = keys_cen.index(c)
            if S[i,j] < m:
                m = S[i,j]
                cluster = c
        nodes[str(key)]['Cluster'] = cluster

    print(centroids)

    i = [0 for i in centroids]

    nonodes = 0
    for node in nodes:
        i[keys_cen.index(nodes[node]['Cluster'])] += 1
        nonodes += 1
    print(np.array(i)*100/nonodes)



In [53]:
import json
with open('./graphs/smallRandom.json', 'r') as f:
    data = json.load(f)

nodes = data['nodes']

kmeans(nodes, 3)
print(nodes)




[ 8.  6. 10.]
[10. 80. 10.]
{'1': {'Data': '00:47:23.6552354', 'Dependencies': [], 'Cluster': 6.0}, '2': {'Data': '01:00:56.0712509', 'Dependencies': [1], 'Cluster': 6.0}, '3': {'Data': '00:45:41.9822376', 'Dependencies': [1], 'Cluster': 6.0}, '4': {'Data': '01:09:26.2243330', 'Dependencies': [1], 'Cluster': 6.0}, '5': {'Data': '01:24:25.7910236', 'Dependencies': [2], 'Cluster': 6.0}, '6': {'Data': '01:25:16.0078169', 'Dependencies': [4], 'Cluster': 6.0}, '7': {'Data': '01:04:38.7031112', 'Dependencies': [2], 'Cluster': 6.0}, '8': {'Data': '00:59:56.1902359', 'Dependencies': [5], 'Cluster': 8.0}, '9': {'Data': '01:27:32.5795340', 'Dependencies': [7], 'Cluster': 6.0}, '10': {'Data': '00:48:03.6393571', 'Dependencies': [8], 'Cluster': 10.0}}
